In [36]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient('localhost', 27017)

db = client['servant_chatbot']

collection = db['CivilComplaints']

docs = collection.find_one({'doc_id': 50597})

pprint(docs)


{'Gu': '양천구',
 '_id': ObjectId('67346cd1953f7e1a68a9c8fc'),
 'answer': '안녕하십니까? 안전교통국장 황광선입니다. 우리 구정에 많은 관심과 애정을 가져주신 귀하께 감사의 말씀드리며, 건의하신 '
           '민원사항에 대한 검토결과를 다음과 같이 알려드립니다. 귀하의 민원은 “신월동(신월3동 주변)~7호선·9호선 연계 '
           '마을버스 노선 신설 요청(1안 : 7호선 까치울역~남부순환로 경유~9호선 마곡나루역 노선 / 2안 : 7호선 '
           '까치울역~남부순환로 경유~신월동 순환노선) ”으로 이해됩니다. 먼저 신월3동 주변에서 가장 가까운 5호선 화곡역이나 '
           '까치산역 쪽으로는 시내버스 노선이 운행하고 있으나 그 외 7호선(까치울역) 또는 9호선(마곡나루역)까지 가는 마을버스가 '
           '없어 불편을 겪으신 점 안타까운 마음을 전합니다. 마을버스는 서울시(버스정책과)에서 준공영제로 운영하는 시내버스와 달리 '
           '개인사업자가 직접 운영하고 있어 우선적으로 노선을 신설하고자 하는 운수회사에서 관련 법, 이용수요, 수익성 등을 '
           '검토하여 자치구로 신청할 경우, 구에서는 관련 법, 서울시 조례 및 지침 등을 검토하고 최종적으로 서울시의 승인을 '
           '받아야 하는 사항입니다. 다만 마을버스를 신설하고자 하는 운수회사(사업자)는 여객자동차운수사업법 등 관련 법에 따라 '
           '버스 7대 이상 등록 및 차고지를 보유해야 하고, 시내버스 중복운행 구간에서는 시내버스 정류소와 4개소까지만 중복 허용 '
           '등 서울시 조례(서울특별시 여객자동차운수사업의 재정지원 및 한정면허 등에 관한 조례) 및 지침의 요건에 부합되어야 하나 '
           '현실적으로 차고지 보유(확보)에 어려움이 많고 시내버스 노선과의 정류소 중복은 피할 수 없는 실정입니다. 

In [31]:
list_docs = collection.find({}, {'doc_id': 1, '_id': 0})
list_docs = list(list_docs)

print(list_docs)
for doc in list_docs:
    print(doc['doc_id'])

[{'doc_id': 51299}, {'doc_id': 51293}, {'doc_id': 51291}, {'doc_id': 51282}, {'doc_id': 51268}, {'doc_id': 51254}, {'doc_id': 51236}, {'doc_id': 51209}, {'doc_id': 51205}, {'doc_id': 51187}, {'doc_id': 51176}, {'doc_id': 51175}, {'doc_id': 51174}, {'doc_id': 51161}, {'doc_id': 51157}, {'doc_id': 51148}, {'doc_id': 51062}, {'doc_id': 51055}, {'doc_id': 51053}, {'doc_id': 51044}, {'doc_id': 51043}, {'doc_id': 51025}, {'doc_id': 50983}, {'doc_id': 50979}, {'doc_id': 50969}, {'doc_id': 50952}, {'doc_id': 50931}, {'doc_id': 50922}, {'doc_id': 50921}, {'doc_id': 50920}, {'doc_id': 50918}, {'doc_id': 50917}, {'doc_id': 50913}, {'doc_id': 50911}, {'doc_id': 50879}, {'doc_id': 50877}, {'doc_id': 50869}, {'doc_id': 50855}, {'doc_id': 50814}, {'doc_id': 50792}, {'doc_id': 50753}, {'doc_id': 50734}, {'doc_id': 50723}, {'doc_id': 50697}, {'doc_id': 50682}, {'doc_id': 50672}, {'doc_id': 50663}, {'doc_id': 50662}, {'doc_id': 50661}, {'doc_id': 50657}, {'doc_id': 50638}, {'doc_id': 50637}, {'doc_id': 

In [11]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)

from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI


from langchain.llms import Ollama

# 1. Ollama LLM 초기화
# llm = Ollama(
#     model="llama3.2-3b-korean-test:latest",  # Ollama에서 사용할 모델 이름
#     base_url="http://localhost:11434",  # Ollama가 실행 중인 기본 URL
#     temperature=0.6,  # 모델의 창의성 설정
# )
api_key = api_key

llm = ChatOpenAI(
        api_key=api_key,
        model="gpt-4o",
        temperature=0.6,
    )

In [37]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.llms import Ollama
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel

# 1. Ollama LLM 초기화
# llm = Ollama(
#     model="llama3.2-3b-korean-test:latest",  # Ollama에서 사용할 모델 이름
#     base_url="http://localhost:11434",  # Ollama가 실행 중인 기본 URL
#     temperature=0.6,  # 모델의 창의성 설정
# )

# 5. 후속 질문
title = docs['title']
content = docs['request_content']
actual_answer = docs['answer']
pprint(content)
pprint(actual_answer)


system_message = SystemMessagePromptTemplate.from_template(
    """You are an expert in analyzing and summarizing civil complaints using systematic reasoning (Chain of Thought, CoT).
Your task is to analyze complaints and create clear and concise summaries while explicitly demonstrating the reasoning steps involved in the process.
You should answer in only Korean using administratively appropriate terms.
모든 분석 및 요약은 행정적으로 적합한 용어를 사용해 한국어로 작성하세요.

분석 과정은 다음과 같은 단계를 포함합니다:
1. 주요 요청 사항 식별
2. 민원 배경 및 상황 요약
3. 현재 상태 및 정책 설명
4. 요청 사유 또는 근거 제시
5. 민원이 원하는 구체적인 결과 도출
6. 핵심 요약

최종적으로, 위 분석을 바탕으로 감정적 표현이나 구체적인 세부 사항을 제거하고 '한 문장'으로 민원의 핵심을 요약하세요. 불만사항 중심 표현을 유지하며, 문제의 원인을 포함한 요약을 작성하세요.

### output
{{
    "key_request": "",
    "background": "",
    "current_state": "",
    "rationale": "",
    "desired_outcome": "",
    "summary": ""
}}
"""
)

user_message = HumanMessagePromptTemplate.from_template(
    """다음은 주민이 제기한 민원입니다:
    '지난 봄에도 도로변 주차 단속 변경을 요청드렸는데 달라진 게 없네요...사람마다 상황과 입장이 다 '
    '다르겠지만 주차할 곳은 없는데 출근시간까지 애매하니 정말 짜증나는데요...현재 7시30분 정도부터 '
    '도로변 주차 단속 하는 걸로 알고 있는데요...출근은 8시쯤이라 허둥지둥 차를 아파트에 올려놨다가 '
    '출근합니다...근데 7시30분쯤에도 아파트에 마땅한 자리가 없는 경우도 있어 당황할 때도 '
    '많습니다...곧 겨울이기도 하니 단속 시간을 8시 이후로 늦춰주시면 어떨까 합니다.'
"""
)

ai_message = AIMessagePromptTemplate.from_template(
    """분석 결과:
1. 주요 요청 사항: 아침 8시 이후로 도로변 주차 단속 시간을 늦춰달라는 요청
2. 배경 및 상황: 출근 시간에 아파트 주차 공간 부족으로 불편을 겪고 있음
3. 현재 상태: 현재 단속 시간은 오전 7시 30분부터 시작됨
4. 요청 근거: 아침 출근 준비로 인한 불편함과 겨울철 상황 고려
5. 원하는 결과: 단속 시작 시간을 오전 8시 이후로 조정
6. 핵심 요약: 아침 출근 시간대 주차 단속을 오전 8시 이후로 늦춰달라는 민원
"""
)

user_message_2 = HumanMessagePromptTemplate.from_template(
    '안녕하세요지금 주소지에 30년가까이 살고 있는 양천구 주민입니다.저희 빌라는 총 5세대로, 주차장은 집 옆쪽으로 4대 세울 수 있는 '
 '공간이 별도로 있습니다.하지만 5세대인만큼 저희 빌라는 돌아가며 집 앞쪽으로 한대를 세우고 있으며, 지금까지 20년 넘게 그 부분으로 '
 '문제가 일어나지 않았습니다,심지어 저희 아버지는 주차공간이 없으니, 차를 먼곳 주차장을 따로 구매해 세우거나 잘 안가져오십니다.다른 '
 '세대들 중에 집에 차가 두대 소유인경우에도 그렇게 하고 있구요.하지만 며칠전에 갑자기 한번도 일어나지 않은 일이 일어났습니다빌라 바로 '
 '앞에 바짝 세운 제 차에 과태료 부과 종이가 올려져 있는 것을 보고 굉장히 낙담하였습니다.집 앞이 어린이보호구역도 아니며, 차를 '
 '주차한다고 해도 소방차 지나갈 수 있는 거리이며심지어 맞은편 빌라에서 주차하시는 차들도 충분히 나올 수 있는 공간 계산해두어 표시해놨고, '
 '피해끼치지 않는 선에서 주차하고 있습니다.구청에 예전에 주차구역 선 그리는것에 대해 문의드렸을때, 규격에 아주 약간 모자르다고 따로 '
 '그것에 대해 해놓을 수는 없다 하셔서 어쩔 수 없이 현상황에 만족하며 살고 있습니다.그런데, 30년 가까이 문제없이 지내다가, 지금까지 '
 '아무런 문제도 싸움도 일어나지 않은 곳에서, 신고전화 한번으로 과태료를 부과해야하는 상황이라니요.어떤분이 어떤이유에서 신고를 하셨는지는 '
 '모르겠지만, 제가 여기 거주하지 않는 주민이라고 생각하신건지 모르겠지만,그 누구보다 오랫동안 이 곳에서 살고 있는 저희 가족, 저희 빌라 '
 '주민으로서는 조금 황당하지 않을 수 없습니다.문제를 일으켰다면 과태료를 낼 수는 있습니다만, 이번에 아무 해결방안 없이 내고 만다면, '
 '똑같이 이런 상황이 발생할 것이고, 저희 빌라는 그럼 한순간에 갑자기 주차공간 한곳을 잃게됩니다. 이렇게 무책임한 일이 '
 '있을까요?신고하신분이 불편하신 점이 있으셨다면 그부분을 논의하고 해결방안을 구청과 저희와 함께 찾아야한다고 생각합니다.그게 아니라 무조건 '
 '과태료 부과 대상 차량이다. 라고 하신다면 저도 굉장히 양천구민이라는것에 답답하고 화가 날것같네요.이의제기 문의드리는 곳으로 전화드려도 '
 '응대도 엉망이고, 그냥 저희 단속 결과에 따르면 그렇네요. 라며 제 잘못으로 똑같은 말씀만 하셔서 굉장히 답답했습니다.이런식의 처리방안은 '
 '옳지 않다고 보며, 제 민원신청에 대해 정확히 해결 부탁드립니다.'
)

ai_message_2 = AIMessagePromptTemplate.from_template(
    """분석 결과:
1. 주요 요청 사항: 이면도로변 불법 주·정차 단속 과태료 부과에 대한 이의를 제기하며, 해결 방안을 요구
2. 배경 및 상황: 민원인은 30년간 해당 지역에 거주하며, 빌라 거주민들 간의 주차 관행이 오랜 기간 동안 문제없이 유지됨.
3. 현재 상태: 신고에 의해 과태료가 부과되었으며, 기존 주차 공간의 사용에 제약이 발생함.
4. 요청 근거: 단속으로 인해 기존 관행에 따른 주차 공간을 잃게 되어 주민 생활에 불편을 초래함. 또한, 신고자의 불편 사항과 해결 방안을 논의할 필요성 제기.
5. 원하는 결과: 과태료 면제 및 주차 문제 해결을 위한 협의와 대안 마련.
6. 핵심 요약: 이면도로변 불법 주·정차 과태료 부과에 대한 이의와 해결 방안을 요청하는 민원.
"""
)

user_message_3 = HumanMessagePromptTemplate.from_template(
    '서울시 교통도시실장 말씀하시는거보니목동선이 예타에서 b/c의 문턱을 넘지 못하고 탈락했네요?총선 지나고 탈락 발표하려고 뜸들이고 '
 '있고..연간 150~200억 정도 적자가 발생하는 것이 주요골자인것 같은데..우리 50만 양천구민이 매월 5천원~1만원씩 걷어서 목동선 '
 '진행합시다.양천구 내에서만 노선이 있으니 크게 반대하시는 분들 없을거라 봅니다.우리 양천구민들이 언제까지 이렇게 대접도 못받고 살아야 '
 '합니까??신월동에는 지하철역이 단 1개도 없습니다.총선 때 "구민투표" 같이 진행하여목동선 착공 하는 것 고려해주세요.우리 양천구민들이 '
 '그 비용 부담하겠습니다. 여건 되시는 분들은 더 내실테니 충분할겁니다.'
)

ai_message_3 = AIMessagePromptTemplate.from_template(
    """분석 결과:

    1. 주요 요청 사항( 이 민원에서 요청하는 가장 중요한 행동 또는 변경은 무엇인가요?): "목동선이 예비타당성 조사에서 경제성 부족으로 탈락한 상황",
    2. 배경 및 상황 ( 이 민원이 발생한 배경과 상황은 무엇인가요?): "목동선이 예타에서 b/c 기준을 넘지 못해 진행이 보류됨",
    3. 현재 상태 (이 민원이 언급하는 현재 상황이나 정책은 무엇인가요?): "양천구 내 지하철역 부족과 주민들이 자발적으로 비용을 부담하겠다는 의사",
    4. 요청 근거 (민원인이 이 요청을 정당화하기 위해 제시한 이유는 무엇인가요?): "구민투표를 통해 목동선 착공을 진행하고 비용을 구민들이 부담",
    5. 원하는 결과 (민원인이 원하는 구체적인 변경 사항 또는 결과는 무엇인가요?):  
    6. 핵심 요약: 목동선 경전철 사업 추진
    """
)

analysis_template = """다음 민원을 분석하고 단계적으로 답변하세요:

제목: {title}
내용: {content}

1. 주요 요청 사항 ( 이 민원에서 요청하는 가장 중요한 행동 또는 변경은 무엇인가요?):
2. 배경 및 상황 ( 이 민원이 발생한 배경과 상황은 무엇인가요?): 
3. 현재 상태 (이 민원이 언급하는 현재 상황이나 정책은 무엇인가요?): 
4. 요청 근거 (민원인이 이 요청을 정당화하기 위해 제시한 이유는 무엇인가요?): 
5. 원하는 결과 (민원인이 원하는 구체적인 변경 사항 또는 결과는 무엇인가요?): 

위 분석을 바탕으로 '한 문장'으로 핵심 요약을 작성하세요.

### output
{{
    "key_request": "",
    "background": "",
    "current_state": "",
    "rationale": "",
    "desired_outcome": "",
    "summary": ""
}}

"""
next_question = HumanMessagePromptTemplate.from_template(analysis_template)

# Define the schema for the output
class ComplaintSchema(BaseModel):
    key_request: str
    background: str
    current_state: str
    rationale: str
    desired_outcome: str
    summary: str

# Create the parser
parser = PydanticOutputParser(pydantic_object=ComplaintSchema)

# cot chain
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message, user_message, ai_message, user_message_2, ai_message_2, user_message_3, ai_message_3, next_question]
)

chain = LLMChain(llm=llm, prompt=chat_prompt)

response = chain.run(title=title, content=content)

result = parser.parse(response)
print('='*50)
print(response)

('강서나 구로 는 마을버스 참 많아요 우리 양천구는 왜 마을버스 노선이 목동 신정동 쪽에만 있나요신월동에 마을버스 신설 건의 합니다 구로구 '
 '강서구는 마을버스 노선이 10대가 넘어 갑니다 신월동은 화곡 이나 까치산쪽으로만 버스노선이 있어서사각 지대입니다 노선 신설이 시급합니다 '
 '1안은 7호선 까치울역과 9호선 마곡나루 노선2안은 7호선 까치울역과 신월동 순환노선 입니다')
('안녕하십니까? 안전교통국장 황광선입니다. 우리 구정에 많은 관심과 애정을 가져주신 귀하께 감사의 말씀드리며, 건의하신 민원사항에 대한 '
 '검토결과를 다음과 같이 알려드립니다. 귀하의 민원은 “신월동(신월3동 주변)~7호선·9호선 연계 마을버스 노선 신설 요청(1안 : 7호선 '
 '까치울역~남부순환로 경유~9호선 마곡나루역 노선 / 2안 : 7호선 까치울역~남부순환로 경유~신월동 순환노선) ”으로 이해됩니다. 먼저 '
 '신월3동 주변에서 가장 가까운 5호선 화곡역이나 까치산역 쪽으로는 시내버스 노선이 운행하고 있으나 그 외 7호선(까치울역) 또는 '
 '9호선(마곡나루역)까지 가는 마을버스가 없어 불편을 겪으신 점 안타까운 마음을 전합니다. 마을버스는 서울시(버스정책과)에서 준공영제로 '
 '운영하는 시내버스와 달리 개인사업자가 직접 운영하고 있어 우선적으로 노선을 신설하고자 하는 운수회사에서 관련 법, 이용수요, 수익성 등을 '
 '검토하여 자치구로 신청할 경우, 구에서는 관련 법, 서울시 조례 및 지침 등을 검토하고 최종적으로 서울시의 승인을 받아야 하는 '
 '사항입니다. 다만 마을버스를 신설하고자 하는 운수회사(사업자)는 여객자동차운수사업법 등 관련 법에 따라 버스 7대 이상 등록 및 차고지를 '
 '보유해야 하고, 시내버스 중복운행 구간에서는 시내버스 정류소와 4개소까지만 중복 허용 등 서울시 조례(서울특별시 여객자동차운수사업의 '
 '재정지원 및 한정면허 등에 관한 조례) 및 지침의 요건에 부합되어야 하나 현실적으로 차고지 보유(확보)에 어려움이 많고 시내버스

In [38]:
result_dict = result.dict()
pprint(result_dict)

summary = result_dict['summary']


{'background': '강서구와 구로구에는 많은 마을버스 노선이 존재하지만, 양천구 신월동에는 마을버스가 부족함',
 'current_state': '신월동의 버스 노선이 화곡이나 까치산 쪽으로만 한정되어 있음',
 'desired_outcome': '신월동에 7호선 까치울역과 9호선 마곡나루를 연결하거나 신월동 순환 마을버스 노선 신설',
 'key_request': '신월동에 마을버스 노선 신설 요청',
 'rationale': '신월동 주민들이 대중교통 사각지대에 놓여 있어 교통 불편을 겪고 있음',
 'summary': '신월동 지역의 대중교통 사각지대를 해소하기 위해 마을버스 노선 신설이 필요함'}


In [39]:
to_cypher_template = """
make the following summary into a cypher query:
### instruction
1. you should ignore any emotional expressions and specific details.
2. you should keep core expressions
3. you should make the output as dictionary format
### Summary
{summary}
### output:
{{
    "summary": "{summary}",
    "cypher": ""
}}
"""

# Define the schema for the output
class CypherSchema(BaseModel):
    summary: str
    cypher : str

to_cypher = HumanMessagePromptTemplate.from_template(template=to_cypher_template, input_variables=["summary"])

chat_prompt = ChatPromptTemplate.from_messages([to_cypher])

cypher_chain = LLMChain(llm=llm, prompt=chat_prompt)

summary = result_dict['summary']

response = cypher_chain.run(summary=summary)

pprint(response)

cypher_parser = PydanticOutputParser(pydantic_object=CypherSchema)

result = cypher_parser.parse(response)

result_dict = result.dict()

pprint(result_dict)


('```json\n'
 '{\n'
 '    "summary": "신월동 지역의 대중교통 사각지대를 해소하기 위해 마을버스 노선 신설이 필요함",\n'
 '    "cypher": "CREATE (Route:BusRoute {name: \'신월동 '
 '마을버스\'})-[:SERVES]->(Area:Neighborhood {name: \'신월동\'})"\n'
 '}\n'
 '```')
{'cypher': "CREATE (Route:BusRoute {name: '신월동 "
           "마을버스'})-[:SERVES]->(Area:Neighborhood {name: '신월동'})",
 'summary': '신월동 지역의 대중교통 사각지대를 해소하기 위해 마을버스 노선 신설이 필요함'}


In [35]:
masking_template = """
You are a helpful assistant. Your task is to mask the names and other private information in the input text.

### instruction
1. Mask all names and other private information in the text.
2. you should keep the original text structure.
3. you should make the output as dictionary format
### input
{input_text}
### output:
{{
    "masked_text": ""
}}
"""

# Define the schema for the output
class MaskingSchema(BaseModel):
    masked_text: str

masking_parser = PydanticOutputParser(pydantic_object=MaskingSchema)

masking_prompt = HumanMessagePromptTemplate.from_template(masking_template, input_variables=["input_text"])

chat_prompt = ChatPromptTemplate.from_messages([masking_prompt])

input_text = """'우리 구정에 많은 관심과 애정을 가져주신 귀하께 감사드리며, 요청하신 민원사항에 대한 검토결과를 다음과 같이 알려드립니다.귀하의 '
 '민원내용은 목동아파트 6단지 주변 불법 주·정차 단속에 대한 불만사항으로 이해됩니다. 귀하의 의견은 잘 읽어보았으며, 먼저 해당 불법 '
 '주·정차 단속으로\xa0\xa0 인해 겪으셨을 불편한 심경에 대해 깊은 위로의 말씀을 드립니다.우리 구에서는 평소 목동아파트 단지의 '
 '심각한 주차난을 고려하여 주말, 공휴일 및 평일 21:00~07:00시 까지 주차단속 완화를 통한 탄력적인 주·정차 단속을 시행하고 '
 '있으며, 주말 및 공휴일은 120 신고, 전화 민원에 의한 현장 출동으로 불법 주·정차 단속을 진행하고 있습니다. 또한 주차단속 강화를 '
 '요청하는 민원과 단속을 지양해 달라는 주민들 간의 의견이 서로 상충하고 있어 주·정차절대금지구역(횡단 보도, 모퉁이, 교차로, 보도, '
 '소화전, 어린이보호구역 등) 위주로 단속을 실시하고 있습니다. 목동아파트 5·6단지 사잇길은 현재 어린이보호구역으로 지정된 도로에 '
 '해당되고\xa0\xa0운행하는 노선버스의 배차 간격이 5분 정도로 짧은 관계로 2020년부터 버스 운수회사 측의 불법 주·정차 단속을 '
 '강화해 달라는 민원 제기가 끊이지 않고 계속되어\xa0\xa0 우리 구에서도 많은 어려움을 겪고 있는 실정임을 이해해 주시기 바랍니다. '
 '최근 운수회사 측과의 협의사항으로 목동아파트 5·6단지 주변 불법 주·정차 차량들에\xa0\xa0대한 경각심 환기를 위해 불가피하게 '
 '해당지역 불법 주·정차 경고장 발부를 시행할 수 밖에 없는 상황임을 말씀드리며, 차후에도 올바른 주차문화 확립에 힘써 주시기를 '
 '부탁드립니다. 귀하의 질문에 만족스러운 답변이 되었기를 바라며, 무더운 날씨에 건강 유의하시기 바랍니다. 답변 내용에 대해 추가로 설명이 '
 '필요한 경우 주차관리과 이인송 주무관(☎02-2620-3745)으로 문의주시면 친절하게 안내 드리겠습니다. 감사합니다.\xa0\xa0'
 '끝.2024. 8. 29.안전교통국장 황광선 드림 (처리부서 : 주차관리과장 정성진)'"""

masking_chain = LLMChain(llm=llm, prompt=chat_prompt)

response = masking_chain.run(input_text=input_text)

result = masking_parser.parse(response)

result_dict = result.dict()

pprint(result_dict)





{'masked_text': '우리 구정에 많은 관심과 애정을 가져주신 귀하께 감사드리며, 요청하신 민원사항에 대한 검토결과를 다음과 같이 '
                '알려드립니다.귀하의 민원내용은 목동아파트 6단지 주변 불법 주·정차 단속에 대한 불만사항으로 이해됩니다. '
                '귀하의 의견은 잘 읽어보았으며, 먼저 해당 불법 주·정차 단속으로 인해 겪으셨을 불편한 심경에 대해 깊은 '
                '위로의 말씀을 드립니다.우리 구에서는 평소 목동아파트 단지의 심각한 주차난을 고려하여 주말, 공휴일 및 평일 '
                '21:00~07:00시 까지 주차단속 완화를 통한 탄력적인 주·정차 단속을 시행하고 있으며, 주말 및 공휴일은 '
                '120 신고, 전화 민원에 의한 현장 출동으로 불법 주·정차 단속을 진행하고 있습니다. 또한 주차단속 강화를 '
                '요청하는 민원과 단속을 지양해 달라는 주민들 간의 의견이 서로 상충하고 있어 주·정차절대금지구역(횡단 보도, '
                '모퉁이, 교차로, 보도, 소화전, 어린이보호구역 등) 위주로 단속을 실시하고 있습니다. 목동아파트 5·6단지 '
                '사잇길은 현재 어린이보호구역으로 지정된 도로에 해당되고 운행하는 노선버스의 배차 간격이 5분 정도로 짧은 '
                '관계로 2020년부터 버스 운수회사 측의 불법 주·정차 단속을 강화해 달라는 민원 제기가 끊이지 않고 계속되어 '
                '우리 구에서도 많은 어려움을 겪고 있는 실정임을 이해해 주시기 바랍니다. 최근 운수회사 측과의 협의사항으로 '
                '목동아파트 5·6단지 주변 불법 주·정차 차량들에 대한 경각심 환기를 위해 불가피하게 해당지역 불법 주·정차 '
                '경고장 발부를 시행할 수 밖에 없는 상황임을 말씀드